# RGB To Gocad Palette

Input: A point set file containing RGB values for each point
Output: A 256 color palette file + the point file with a texture coordinate column

This notebook loads a set of RGB values and performs K-means in RGB space to get the most representative colors.
Part of this notebook is inspired by https://victorangeloblancada.github.io/blog/2019/10/01/color-palette-clustering.html

### First, some imports and global variables

In [8]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
# from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans

# Fix random seed
np.random.seed(0)

# Set path to read / store files
# from pathlib import Path
path='D:\Data\Field\FOLD_CAQUERELLE\LiDAR3Dpoint clouds\\'
input_file='Leica2_250413.ptx'
base_name = input_file.rsplit('.', 1)[0]

In [9]:
### Reading the input file

In [10]:
def read_leica_lidar( input_leica_ptx_file ) :
    leica_data = pd.read_csv(
        path + input_leica_ptx_file, delimiter = ' ', header=10,
        names=('X', 'Y', 'Z', 'Intensity', 'R', 'G', 'B' )
    )
    return leica_data

data = read_leica_lidar(input_file)

### Run the rgb clustering to form the color palette

In [11]:
# kmeans = KMeans(n_clusters=16, random_state=0)  # Too slow
kmeans = MiniBatchKMeans( n_clusters=256, random_state=0, batch_size=2048)

data['Texture'] = kmeans.fit_predict(data[['R','G','B']])

data

,X,Y,Z,Intensity,R,G,B,Texture
0,10.643173,-10.056595,-1.274948,0.388220,54,58,43,61
1,10.763077,-10.169998,-1.276779,0.400244,60,66,52,32
2,10.770676,-10.177170,-1.266159,0.398413,60,66,52,32
3,10.783310,-10.189133,-1.256271,0.410010,57,65,50,32
4,10.963943,-10.360001,-1.264145,0.348608,53,61,46,126
...,...,...,...,...,...,...,...,...
3049743,-19.797928,-22.150620,20.661575,0.324727,56,45,31,68
3049744,-19.818619,-22.173904,20.719223,0.248493,53,41,25,196
3049745,-19.853714,-22.213303,20.791428,0.240803,64,53,35,40
3049746,-19.848343,-22.207382,20.820358,0.241962,80,70,45,157


In [12]:
palette = kmeans.cluster_centers_

### Create the color map file in Gocad 2019 format

In [13]:
def write_palette_file( color_map ):
    file = open(path + base_name + '.xcmap', 'w+')
    file.write("""<?xml version="1.0" encoding="UTF-8" standalone="no" ?>
<Colormaps xmlns="http://pdgm.com/Gocad/ColormapEntity/2.8.0">
    <ColormapEntity>
""")
    file.write( f'<Name> {base_name} </Name>')
    file.write("""
        <Kind>linear</Kind>
        <Colors>
        """)
    color_index = 0
    for col in color_map :
        file.write( f'<Color index=" {color_index} " rgb="' +
                    f'{col[0]/255} {col[1]/255} {col[2]/255}"/>\n'
                    )
        color_index+=1
    file.write("""
            </Colors>
            </ColormapEntity>
        </Colormaps>
        """)

### Write the output files

In [ ]:
data.to_csv(path + base_name + '.csv',
            sep = ' ',
            columns=['X','Y','Z','Intensity','Texture'],
            index=False)
write_palette_file(palette)

